In [2]:
import numpy as np
import sys
import pathlib
sys.path.append('../../../build')
import libry as ry
import time

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/02-kinematics/solutions'
** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}
** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/course3-Simulation/02-kinematics/solutions'


** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}



## Circling motion with coordinate transformation 

The circling motion can be easily made in the polar cordinate, e.g., $$\dot{r} = 1-r,~\dot{\theta}=1.$$
So, all we need to do is:
1. transform the feature value (again only $(y_1,y_3)$ components, with $\dot{y}_2=-y_2$) into polar coordinate, $$r=\sqrt{y_1^2+y_3^2},~\theta=\arctan2(y_1,y_3).$$
2. compute the desired velocity in polar coordinate, $$\dot{r} = 1-r,~\dot{\theta}=1.$$
3. transform it back to the original feature space, $$\dot{y}_1 = \dot{r}\cos\theta-r\dot{\theta}\sin\theta,\\
\dot{y}_3 = \dot{r}\sin\theta+r\dot{\theta}\cos\theta.$$
(derived from $y_1=r\cos\theta, y_3=r\sin\theta$)
4. take the psuedoinverse to get the joint velocity.

### My notes:
r is the distance between the frame center and the object (radius 1 is desired)


In [2]:
#-- MODEL WORLD configuration, this is the data structure on which you represent
# what you know about the world and compute things (controls, contacts, etc)
C = ry.Config()
C.addFile( str(path / "scenarios/pandasTable.g" ))
D = C.view()
q0 = C.getJointState()

In [3]:
obj = C.addFrame("object")
# set frame parameters, associate a shape to the frame, 
pos_obj = np.array([.5,0,1.5])
pos_obj0 = pos_obj.copy()
obj.setPosition(pos_obj)
obj.setQuaternion([1,0,0,0])
obj.setShape(ry.ST.sphere, [.02])
obj.setColor([1,0,0])

In [ ]:
q = C.getJointState()
print('joint names: ', C.getFrameNames() )
print('joint state: ', q)

In [4]:
tau = .01
C.setJointState(q0)
q = C.getJointState()
for t in range(1000):
    time.sleep(0.01)
    
    #### Solution
    
    [y,J] = C.evalFeature(ry.FS.positionRel, ["R_gripperCenter", "object"])
    # pos_obj[0] = 1 - np.sin(0.01*t) #move the obj for fun!
    obj.setPosition(pos_obj)

    # 1. transform to polar coordinate
    r = np.sqrt(y[0]**2 + y[2]**2)
    theta = np.arctan2(y[2],y[0])

    # 2. compute the velocity there (you can change the params of dynamics)
    r_dot = 0.2-r
    theta_dot = -10.

    # 3. transform back to feature space (gripper velocity)
    y1_dot = r_dot*np.cos(theta)-r*theta_dot*np.sin(theta)
   
    y3_dot = r_dot*np.sin(theta)+r*theta_dot*np.cos(theta)
    y2_dot = -y[1]
    
    # alternative ODE that generates circles
    # (kepping the initial radius)
    #y1_dot = -3*y[2]
    #y3_dot = 3*y[0]
    #y2_dot = -y[1]

    
    

    # 4. take the psuedoinverse to get the joint velocity.
    vel_ee = np.array([y1_dot, y2_dot, y3_dot])
    
    ####
    
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ vel_ee;
    
    q += tau*vel
    C.setJointState(q) 

KeyboardInterrupt: 